In [1]:
!pip install tensorflow-gpu

    100% |████████████████████████████████| 345.2MB 84kB/s 


In [2]:
!apt-get install -y -qq software-properties-common python-software-properties module-init-tools
!add-apt-repository -y ppa:alessandro-strada/ppa 2>&1 > /dev/null
!apt-get update -qq 2>&1 > /dev/null
!apt-get -y install -qq google-drive-ocamlfuse fuse
from google.colab import auth
auth.authenticate_user()
from oauth2client.client import GoogleCredentials
creds = GoogleCredentials.get_application_default()
import getpass
!google-drive-ocamlfuse -headless -id={creds.client_id} -secret={creds.client_secret} < /dev/null 2>&1 | grep URL
vcode = getpass.getpass()
!echo {vcode} | google-drive-ocamlfuse -headless -id={creds.client_id} -secret={creds.client_secret}

E: Package 'python-software-properties' has no installation candidate
Selecting previously unselected package google-drive-ocamlfuse.
(Reading database ... 131304 files and directories currently installed.)
Preparing to unpack .../google-drive-ocamlfuse_0.7.3-0ubuntu3~ubuntu18.04.1_amd64.deb ...
Unpacking google-drive-ocamlfuse (0.7.3-0ubuntu3~ubuntu18.04.1) ...
Setting up google-drive-ocamlfuse (0.7.3-0ubuntu3~ubuntu18.04.1) ...
Processing triggers for man-db (2.8.3-2ubuntu0.1) ...
Please, open the following URL in a web browser: https://accounts.google.com/o/oauth2/auth?client_id=32555940559.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive&response_type=code&access_type=offline&approval_prompt=force
··········
Please, open the following URL in a web browser: https://accounts.google.com/o/oauth2/auth?client_id=32555940559.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=ht

In [0]:
!mkdir -p drive
!google-drive-ocamlfuse drive

In [0]:
import os
os.chdir("drive/colab")

In [5]:
!wget https://bin.equinox.io/c/4VmDzA7iaHb/ngrok-stable-linux-amd64.zip
!unzip ngrok-stable-linux-amd64.zip

--2019-04-22 15:19:57--  https://bin.equinox.io/c/4VmDzA7iaHb/ngrok-stable-linux-amd64.zip
Resolving bin.equinox.io (bin.equinox.io)... 52.203.102.189, 52.21.103.149, 35.172.177.65, ...
Connecting to bin.equinox.io (bin.equinox.io)|52.203.102.189|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 14977695 (14M) [application/octet-stream]
Saving to: ‘ngrok-stable-linux-amd64.zip.1’

ngrok-stable-linux- 100%[===================>]  14.28M  9.02MB/s    in 1.6s    

2019-04-22 15:20:03 (9.02 MB/s) - ‘ngrok-stable-linux-amd64.zip.1’ saved [14977695/14977695]

Archive:  ngrok-stable-linux-amd64.zip
replace ngrok? [y]es, [n]o, [A]ll, [N]one, [r]ename: y
  inflating: ngrok                   


In [6]:
import tensorflow as tf
import os
import pickle
import numpy as  np

CIFAR_DIR = "cifar-10-batches-py"
print(os.listdir(CIFAR_DIR))

#thensorBoard
#1,指定面板图上显示的变量
#2.在训练过程中将这些变量值计算出来写到文件中
#3，文件解析 ./tensorboard -- logdir =dir

def load_data(filename):
    """ read data from data file."""
    with open(filename,'rb') as f:
        data = pickle.load(f ,encoding='latin1') #,encoding='latin1',encoding='iso-8859-1'，encoding='bytes'
        return data['data'],data['labels']
# tensorflow.dataset(可以使用)
class cifardata:
    def __init__(self,filenames,need_shuffle):
        all_data = []
        all_labels=[]
        for filename in filenames:
            data, labels = load_data(filename)
            all_data.append(data)
            all_labels.append(labels)

        self._data =np.vstack(all_data)
        self._data = self._data/127.5 - 1 #数据归一化
        self._labels = np.hstack(all_labels)
        self._num_examples = self._data.shape[0]
        self._need_shuffle = need_shuffle
        self._indicator = 0
        if self._need_shuffle:
            self._shuffle_data()
    def _shuffle_data(self):
        p = np.random.permutation(self._num_examples)
        self._data = self._data[p]
        self._labels = self._labels[p]
    def next_batch(self,batch_size):
        '''return batch_size examples as a batch.'''
        end_indicator = self._indicator + batch_size
        if end_indicator > self._num_examples:
            if self._need_shuffle:
                self._shuffle_data()
                self._indicator = 0
                end_indicator = batch_size
            else:
                raise Exception("have no more examples")
        if end_indicator > self._num_examples:
            raise Exception("batch size is larger than all examples")
        batch_data = self._data[self._indicator:end_indicator]
        batch_lables = self._labels[self._indicator:end_indicator]
        self._indicator = end_indicator
        return batch_data,batch_lables

train_filenames = [os.path.join(CIFAR_DIR,'data_batch_%d' % i ) for i in range(1,6)]
test_filenames  = [os.path.join(CIFAR_DIR,'test_batch')]

train_data = cifardata(train_filenames,True)
test_data = cifardata(test_filenames,False)



x = tf.placeholder(tf.float32, [None,3072])
y = tf.placeholder(tf.int64,[None])
x_image = tf.reshape(x,[-1,3,32,32])
x_image = tf.transpose(x_image,perm=[0,2,3,1])
#feature_map ,输出图 ，神经元图
conv1_1 = tf.layers.conv2d(x_image,
                         32, #out channel number
                         (3,3),#kernal_size
                         padding='same',
                         activation=tf.nn.relu,
                         name = 'conv1_1'
                         )
conv1_2 = tf.layers.conv2d(conv1_1,
                         32, #out channel number
                         (3,3),#kernal_size
                         padding='same',
                         activation=tf.nn.relu,
                         name = 'conv1_2'
                         )
#16*16
pooling1 = tf.layers.max_pooling2d( conv1_2,
                                    (2,2),#kernal_size
                                    (2,2),#stride
                                    name='pool1'
                                     )
conv2_1 = tf.layers.conv2d(pooling1,
                         32, #out channel number
                         (3,3),#kernal_size
                         padding='same',
                         activation=tf.nn.relu,
                         name = 'conv2_1'
                         )
conv2_2 = tf.layers.conv2d(conv2_1,
                         32, #out channel number
                         (3,3),#kernal_size
                         padding='same',
                         activation=tf.nn.relu,
                         name = 'conv2_2'
                         )
#8*8
pooling2 = tf.layers.max_pooling2d( conv2_2,
                                    (2,2),#kernal_size
                                    (2,2),#stride
                                    name='pool2'
                                     )
conv3_1 = tf.layers.conv2d(pooling2,
                         32, #out channel number
                         (3,3),#kernal_size
                         padding='same',
                         activation=tf.nn.relu,
                         name = 'conv3_1'
                         )
conv3_2 = tf.layers.conv2d(conv3_1,
                         32, #out channel number
                         (3,3),#kernal_size
                         padding='same',
                         activation=tf.nn.relu,
                         name = 'conv3_2'
                         )
#4*4
pooling3 = tf.layers.max_pooling2d( conv3_2,
                                    (2,2),#kernal_size
                                    (2,2),#stride
                                    name='pool3'
                                     )
#[none , 4*4*32]
flatten = tf.layers.flatten(pooling3)
y_ = tf.layers.dense(flatten,10)


loss = tf.losses.sparse_softmax_cross_entropy(labels=y,logits=y_)
#y_ ->sofmax
#y ->one_hot
#loss->ylogy_
predict = tf.argmax(y_,1)
correct_prediction = tf.equal(predict,y)
accuracy = tf.reduce_mean(tf.cast(correct_prediction,tf.float64))
with tf.name_scope('train_op'):
    train_op = tf.train.AdamOptimizer(1e-3).minimize(loss)

# def variable_summary(var,name):
#      with tf.name_scope(name):
#          mean = tf.reduce_mean(var)
#          with tf.name_scorp('stddev'):
#              stddev =tf.sqrt(tf.reduce_mean(tf.square(var - mean)))
#          tf.summary.scalar('mean',mean)
#          tf.summary.scalar('stddev',stddev)
#          tf.summary.scalar('min',tf.reduce_min(var))
#          tf.summary.scalar('max',tf.reduce_max(var))
#          tf.summary.scalar('histogram',var)
#
# with tf.name_scope('summary'):
#     variable_summary(conv1_1,'conv1_1')
#     variable_summary(conv1_2,'conv1_2')
#     variable_summary(conv2_1,'conv2_1')
#     variable_summary(conv2_2,'conv2_2')
#     variable_summary(conv3_1,'conv3_1')
#     variable_summary(conv3_2,'conv3_2')


loss_summary = tf.summary.scalar('loss',loss)
accuracy_summary = tf.summary.scalar('accuracy',accuracy)
source_image =(x_image +1 )*127.5
inputs_summary = tf.summary.image('input_image',source_image)
merged_summary = tf.summary.merge_all()
merged_summary_test = tf.summary.merge([loss_summary,accuracy_summary])

LOG_DIR='.'
run_label = 'run_vgg_tensorboard'
run_dir = os.path.join(LOG_DIR,run_label)
if not os.path.exists(run_dir):
    os.mkdir(run_dir)
train_log_dir = os.path.join(run_dir,'train')
test_log_dir = os.path.join(run_dir,'test')
if not os.path.exists(train_log_dir):
    os.mkdir(train_log_dir)
if not os.path.exists(test_log_dir):
    os.mkdir(test_log_dir)


init = tf.global_variables_initializer()
batch_size = 20
train_steps = 10000
test_steps = 100
output_summary_every_steps = 100
with tf.Session() as sess:
    sess.run(init)
    train_writer = tf.summary.FileWriter(train_log_dir,sess.graph)
    test_writer = tf.summary.FileWriter(test_log_dir)

    fixed_test_batch_data,fixed_test_batch_labels \
       = test_data.next_batch(batch_size)
    for i in range(train_steps):
        batch_data,batch_labels = train_data.next_batch(batch_size)
        eval_ops = [loss,accuracy,train_op]
        should_output_summary =((1+i) % output_summary_every_steps ==0)
        if should_output_summary:
            eval_ops.append(merged_summary)

        eval_ops_results = sess.run(
            eval_ops,
            feed_dict={
                x:batch_data ,
                y:batch_labels })
        loss_val,acc_val = eval_ops_results[0:2]
        if should_output_summary:
            train_summary_str = eval_ops_results[-1]
            train_writer.add_summary(train_summary_str,i+1)
            test_summary_str = sess.run([merged_summary_test],
                                        feed_dict={
                                            x:fixed_test_batch_data,
                                            y:fixed_test_batch_labels,
                                        })[0]
            test_writer.add_summary(test_summary_str,i+1)

        if (i+1) % 100 == 0:
            print('[train] step: %d ,loss:%4.5f, acc: %4.5f' %(i,loss_val,acc_val))
        if (i+1) % 100 == 0:
            test_data = cifardata(test_filenames, False)
            all_test_acc_val = [ ]
            for j in range(test_steps):
                test_batch_data, test_batch_labels \
                    = test_data.next_batch(batch_size)
                test_acc_val = sess.run(
                    [accuracy],
                    feed_dict={
                        x:test_batch_data,
                        y:test_batch_labels})
                all_test_acc_val.append(test_acc_val)
            test_acc = np.mean(all_test_acc_val)
            print('[test]step: %d ,acc:%4.5f '%(i+1,test_acc ) )

['batches.meta', 'readme.html', 'data_batch_1', 'data_batch_2', 'data_batch_3', 'data_batch_4', 'data_batch_5', 'test_batch']
Instructions for updating:
Use keras.layers.conv2d instead.
Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Use keras.layers.max_pooling2d instead.
Instructions for updating:
Use keras.layers.flatten instead.
Instructions for updating:
Use keras.layers.dense instead.
Instructions for updating:
Use tf.cast instead.
[train] step: 99 ,loss:2.21124, acc: 0.00000
[test]step: 100 ,acc:0.26000 
[train] step: 199 ,loss:1.43547, acc: 0.55000
[test]step: 200 ,acc:0.33500 
[train] step: 299 ,loss:1.58849, acc: 0.45000
[test]step: 300 ,acc:0.34800 
[train] step: 399 ,loss:1.68459, acc: 0.30000
[test]step: 400 ,acc:0.37700 
[train] step: 499 ,loss:1.94276, acc: 0.25000
[test]step: 500 ,acc:0.41100 
[train] step: 599 ,loss:1.25331, acc: 0.50000
[test]step: 600 ,acc:0.41950 
[train] step: 699 ,loss:1.72073, acc: 0.30000
[test]

In [0]:
LOG_DIR = './run_vgg_tensorboard'
get_ipython().system_raw(
'tensorboard --logdir {} --host 0.0.0.0 --port 6006 &'
.format(LOG_DIR)
)

In [0]:
get_ipython().system_raw('./ngrok http 6006 &')

In [9]:
! curl -s http://localhost:4040/api/tunnels | python3 -c \
"import sys, json; print(json.load(sys.stdin)['tunnels'][0]['public_url'])"

https://baea12b3.ngrok.io
